<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [932]:
import os
import sys
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import psycopg2

In [933]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [934]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [935]:
# текст запроса
query_3_1 = f'''
                select 
                count(id)
                from public.vacancies
            '''

In [936]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [937]:
# текст запроса
query_3_1 = f'''
                select 
                count(id)
                from public.employers
            '''

In [938]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [939]:
# текст запроса
query_3_1 = f'''
                select 
                count(id)
                from public.areas
            '''

In [940]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [941]:
# текст запроса
query_3_1 = f'''
                select 
                count(id)
                from public.industries
            '''

In [942]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,294


***

In [943]:
# выводы по предварительному анализу данных


Данные представляют собой сведения в виде **5** таблиц: *Данные по вакансиям, Городам, Работодателям, Сферам деятельности работодателей, Связь работодателей и сфер их деятельности*. Всего представлены сведения об около **50 тыс** вакансий из **1361 региона** (*довольно широкое представительство*). Количество **работодателей** представляет из себя почти **половину от количества вакансий**. Количество **работадателей** примерно **равно количеству сфер деятельности**.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [944]:
# текст 
query_3_1 = f'''
            SELECT
            count(v.id) cnt,
            a.name area
            from public.vacancies v
            join public.areas a on a.id = v.area_id
            group by 2
            order by 1 desc
            limit 5
            '''

In [945]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [946]:
# текст запроса
query_3_1 = f'''
    SELECT
    count(id) cnt
    from public.vacancies
    where salary_from is not null
    or salary_to is not null
            '''

In [947]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [948]:
# текст запроса
query_3_1 = f'''
SELECT
round(avg(salary_from)) low,
round(avg(salary_to)) high
from public.vacancies
            '''

In [949]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,low,high
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [950]:
# текст запроса
query_3_1 = f'''
SELECT
count(id),
schedule,
employment
from public.vacancies
group by 2, 3
order by 1 desc
offset 1 limit 1
            '''

In [951]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count,schedule,employment
0,7802,Удаленная работа,Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [952]:
# текст запроса
query_3_1 = f'''
SELECT
experience,
count(id)
from public.vacancies
where experience is not null
group by 1
order by 2 asc
            '''

In [953]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [954]:
# выводы по детальному анализу вакансий

* С большим отрывом **по количеству вакансий** предсказуемо лидирует **Москва** (5333), далее следуют: *Санкт-Петербург (2851), Минск (2112), Новосибирск (2006), Алматы (1892)*.

* Примерно у **половины** вакансий *представлены сведения о зарплате*.

* Среднее значение **зарплатной вилки: От - 71065, До - 110537**.

* По графику работы с сильным преимуществом лидирует **"Полный день/Полная занятость" (35,367)**, далее - **Удаленная работа/Полная занятость (7,802)**. Остальные графики не сильно отличаются по показателям.

* Наиболее популярное значение по показателю **"Требуемый опыт работы" - "От 1 года до 3 лет" (26152), на 2 месте - "От 3 до 6 лет"	(14511)**, далее - "Нет опыта" (7197) и "Более 6 лет"	(1337).


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [955]:
# текст запроса
query_3_1 = f'''
SELECT
count(v.id),
e.name
from public.vacancies v
join public.employers e on v.employer_id = e.id
group by 2
order by 1 desc
            '''

In [956]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df.head(5)

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1871375525.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [957]:
# текст запроса
query_3_1 = f'''
select
a.name,
count(e.id) "Количество работодателей",
count(v.id) "Количество вакансий"
from public.areas a
left join public.employers e on a.id = e.area
left join public.vacancies v on a.id = v.area_id
where v.id is null
group by 1
order by 2 desc
            '''

In [958]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df.head(5)

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1871375525.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,name,Количество работодателей,Количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [959]:
# текст запроса
query_3_1 = f'''
select
e.name,
count(distinct v.area_id)
from public.employers e
left join public.vacancies v on e.id = v.employer_id
where v.id is not null
group by 1
order by 2 desc
            '''

In [960]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df.head(5)

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1871375525.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [961]:
# текст запроса
query_3_1 = f'''
select
count(e.id)
from public.employers e
left join public.employers_industries i on e.id = i.employer_id
where i.industry_id is null
            '''

In [962]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df.head(5)

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1871375525.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [963]:
# текст запроса
query_3_1 = f'''
select
e.name
from public.employers e
left join public.employers_industries i on e.id = i.employer_id
group by 1
having count(i.industry_id) = 4
order by 1 asc
            '''

In [964]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df.head(3)

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\10312627.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,name
0,101 Интернет
1,21vek.by
2,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [965]:
# текст запроса
query_3_1 = f'''
select
count(i.employer_id)
from public.employers_industries i
join public.industries ind on ind.id = i.industry_id
where ind.name = 'Разработка программного обеспечения'
            '''

In [966]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [967]:
# код для получения списка городов-милионников
import requests 
from bs4 import BeautifulSoup 

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' 
response = requests.get(url)
# дальше работа с содержимым страницы
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
data = []
HTML_data = page.find_all("table")[0].find_all("tr")[1:] # выделяем табличные данные
for element in HTML_data:
    sub_data = []
    for i, k in enumerate(element):
        try:
            if i == 1:
                sub_data.append(k.get_text())
        except:
            continue
    data.append(sub_data)
data_result = [] # Из в спсика списков делаем 1 список
for elem in data:
    data_result.extend(elem)
data_result = tuple(data_result)


In [968]:
# текст запроса
query_3_1 = f'''
with temp as (select
a.name,
count(v.id) cnt
from vacancies v
left join employers e on v.employer_id = e.id
left join areas a on a.id = v.area_id
where a.name in {data_result}
and e.name = 'Яндекс'
and v.id is not null
group by 1
order by 2)
select *
from temp
union all
select
'total',
sum(cnt)
from temp
            '''

In [969]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

In [970]:
# выводы по анализу работодателей

* В первой пятерке работодателей по количеству вакансий с большим преимуществом лидирует **Яндекс** (*1933*), остальные примерно равны.
* Наибольшее количество работадателей среди регионов, где нет вакансий, представлено в **России** (410), на втором месте - **Казахстан** (207), далее идут регионы России. Не вижу большой пользы в данном запросе.
* Наибольшую активность по количеству охваченных вакансиями регионов России проявляет **Яндекс** (181), также высокие показатели у **Ростелекома** (152). Далее идет **Спецремонт и *предпреимчивый* Поляков Д.И.**
* Всего у **8419** работодателей **не указана сфера деятельности**, что довольно высокий показатель и, вероятно, влияет на проявляемый интерес.
* На **3 месте** среди компаний, у которых указано 4 сферы деятельности, находится **2ГИС**.
* **3553** работадателя указали в качестве сферы деятельности "*Разработка программного обеспечения*"
* У компании **Яндекс** из городов-миллионников больше всего **вакансий** представлено в **Москве (54) и Санкт-Петербурге (42)**. Далее идут Екатеринбург, Нижний Новгород, Новосибирск, Воронеж, Краснодар и др. **Всего** компанией Яндекс в городах-миллионниках размещено **485 вакансий**.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [971]:
# текст запроса
query_3_1 = f'''
select
count(id)
from public.vacancies
where lower(name) like '%data%' or lower(name) like '%данн%'
            '''

In [972]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [973]:
# текст запроса
query_3_1 = f'''
select
count(id)
from public.vacancies
where (lower(name) like '%data scientist%'
or lower(name) like '%data science%'
or lower(name) like '%исследователь данных%'
or (upper(name) like '%ML%' and upper(name) not like '%HTML%')
or lower(name) like '%machine learning%'
or lower(name) like '%машинн%обучен%')
and (lower(name) like '%junior%'
or experience = 'Нет опыта'
or employment = 'Стажировка')
            '''

In [974]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [975]:
# текст запроса
query_3_1 = f'''
select
count(id)
from public.vacancies
where (lower(name) like '%data scientist%'
or lower(name) like '%data science%'
or lower(name) like '%исследователь данных%'
or (upper(name) like '%ML%' and upper(name) not like '%HTML%')
or lower(name) like '%machine learning%'
or lower(name) like '%машинн%обучен%')
and (upper(key_skills) like '%SQL%'
or lower(key_skills) like '%postgres%')
            '''

In [976]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [977]:
# текст запроса
query_3_1 = f'''
select
count(id)
from public.vacancies
where (lower(name) like '%data scientist%'
or lower(name) like '%data science%'
or lower(name) like '%исследователь данных%'
or (upper(name) like '%ML%' and upper(name) not like '%HTML%')
or lower(name) like '%machine learning%'
or lower(name) like '%машинн%обучен%')
and lower(key_skills) like '%python%'
            '''

In [978]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [979]:
# текст запроса
query_3_1 = f'''
select 
round(avg(length(key_skills)-length(replace(key_skills,CHR(9),''))+1), 2)
from public.vacancies
where key_skills is not null
and (name ilike '%data scientist%'
or name ilike '%data science%'
or name ilike '%исследователь данных%'
or (name like '%ML%' and name not ilike '%HTML%')
or name ilike '%machine learning%'
or name ilike '%машинн%обучен%')
            '''

In [980]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [981]:
# текст запроса
query_3_1 = f'''
SELECT 
ROUND(AVG(coalesce((salary_from + salary_to) / 2, salary_from, salary_to)), 0),
experience
FROM public.vacancies   
WHERE  ((name ILIKE '%data scientist%') OR (name ILIKE '%data science%') OR (name ILIKE '%исследователь данных%')
    OR ((name LIKE '%ML%')
    AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%'))
and (salary_to is not null or salary_from is not null)
group by 2
            '''

In [982]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\ceasa\AppData\Local\Temp\ipykernel_21704\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,round,experience
0,74643.0,Нет опыта
1,139675.0,От 1 года до 3 лет
2,243115.0,От 3 до 6 лет


***

In [983]:
# выводы по предметному анализу

Установлено, что **1771** вакансия имеет отношение к данным. Для **начинающих Data Scientist** размещена **51** вакансия. Для занаятия должности в DS **SQL или postgres** требуется в **229** случаях. Ключевой навык Data Scientist в виде **Python требуется в 357 случаях**. *В среднем* указывают в вакансиях для DS **6,41 ключевых навыков**. **Зарплатная вилка** в DS варьируется, в т.ч., в зависимости от опыта работы. В среднем *без опыта* работы специалист получает **74643.0** руб, с опытом *от 1 до 3 лет* - **139675*, *от 3 до 6 лет* - **243115**.

# Общий вывод по проекту

In [984]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования


Данные представляют сведения о спросе и предложении **на IT рынке**, преимущественно, России. Представлено около **50 тыс** вакансий из **1361 региона**, есть конкретные критерии к сосискателю со стороы работодателей. Данных **достаточно**, чтобы вывести закономерности по результатам анализа.

С большим отрывом по различным показателям данных **лидирует Москва**: *больше всего вакансий, выше средняя зарплата, большее количество работодателей*. В целом, крупные IT компании лидируют на рынке. Особенно **выделяется "Яндекс"**. Среднее значение **зарплатной вилки: От - 71065, До - 110537**. При этом только у **половины** вакансий *представлены сведения о зарплате*, что, вероятно, говорит о желании работодателя сделать предложение по зарплате по результатам собеседований.

Работодатель в большей степени заинтересован в работниках, которые готовы работать **"Полный день/Полная занятость"**. Подавляющее большинство вакансий требует **наличие опыта работы**. Вместе с тем, работодатель готов брать незначительное количество новичков, в т.ч. в Data Science. Для трудоустройства **требуется занание Python, SQL или postgres**, а также 2-3 ключевых навыка. Средняя начальная зарплата в этом случае составляет около **74 тыс рублей**.

